In [ ]:
#| eval: false
! [ -e /content ] && pip install -Uqq xcube # upgrade xcube on colab

In [ ]:
#| default_exp l2r.data.boot

In [ ]:
#| export
# from fastai.torch_imports import *
# from fastai.data.load import DataLoader
# from xcube.imports import *

In [ ]:
#| export
from fastcore.basics import *
from fastai.torch_core import *
from fastai.data.core import *
from fastai.data.transforms import *
from fastai.text.core import *
from fastai.text.data import *
from xcube.imports import *

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Boot L2R 

> Bootstrapping a learning-to-rank model

This module contains all the functions needed to bootstrap a L2R model from xml text data.

In [ ]:
#| export
def onehot_dsets(df):
    x_tfms = [Tokenizer.from_df('text', n_workers=num_cpus()), attrgetter("text"), Numericalize(), OneHotEncode()]
    y_tfms = [ColReader('labels', label_delim=';'), MultiCategorize(), OneHotEncode()]
    tfms = [x_tfms, y_tfms]
    dsets = Datasets(df, tfms=[x_tfms, y_tfms])
    return dsets

In [ ]:
#| export
class BatchLbsChunkify(ItemTransform):
    order = 100
    def __init__(self, chnk_st, chnk_end): store_attr('chnk_st,chnk_end')
    def encodes(self, x): 
        return (x[0], x[1][:, self.chnk_st:self.chnk_end])

In [ ]:
#| export
def lbs_chunked(dsets, bs=8, chnk_sz=200, device=None):
    lbs = dsets.vocab[1]
    dls = []
    for chnk_st in range(0, len(lbs), chnk_sz):
        dls.append(TfmdDL(dsets, bs=bs, 
                          after_batch=[BatchLbsChunkify(chnk_st, min(chnk_st+chnk_sz, len(lbs)))], 
                          device=default_device() if device is None else device))
    return dls

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()